In [ ]:
#!/usr/bin/env python
# coding: utf-8

import numpy as np

def simulate_oscillator():
    # Parameters
    m = 1.0
    c = 0.2
    k = 4.0
    x0 = 1.0
    v0 = 0.0
    dt = 0.001
    T = 10.0

    # Time grid
    N = int(T / dt)
    t = np.linspace(0, T, N)  # BUG: should include endpoint (N+1)

    # Initialize arrays
    x = np.zeros_like(t)
    v = np.zeros_like(t)
    x[0] = x0
    v[0] = v0

    # Incorrect integration (uses simple Euler instead of RK4)
    for i in range(len(t) - 1):
        a = -(c / m) * v[i] - (k / m) * x[i]
        v[i + 1] = v[i] + a * dt
        x[i + 1] = x[i] + v[i] * dt

    # Wrong analytical solution (forgets damping exponential factor)
    omega_n = np.sqrt(k / m)
    zeta = c / (2 * np.sqrt(k * m))
    omega_d = omega_n * np.sqrt(1 - zeta**2)
    A = x0
    B = (v0 + zeta * omega_n * x0) / omega_d
    # BUG: missing exponential decay term
    x_analytic = A * np.cos(omega_d * t) + B * np.sin(omega_d * t)

    # Error metric
    max_abs_error = np.max(np.abs(x - x_analytic))

    # Energy calculation
    E = 0.5 * m * v**2 + 0.5 * k * x**2
    # BUG: may divide by zero if first E=0 (use safe check in final)
    energy_ratio_end = E[-1] / E[0] if E[0] != 0 else np.nan

    # FFT dominant frequency
    fs = 1.0 / dt
    X = np.fft.fft(x)
    freqs = np.fft.fftfreq(len(x), d=dt)
    # BUG: includes DC and uses full FFT instead of real FFT
    idx = np.argmax(np.abs(X)**2)
    peak_freq = freqs[idx]

    return {
        "t": t,
        "x": x,
        "v": v,
        "max_abs_error": float(max_abs_error),
        "energy_ratio_end": float(energy_ratio_end),
        "peak_freq": float(peak_freq)
    }

# Run simulation
result = simulate_oscillator()
print("Simulation result (buggy):")
print({
    "len_t": len(result["t"]),
    "len_x": len(result["x"]),
    "len_v": len(result["v"]),
    "max_abs_error": result["max_abs_error"],
    "energy_ratio_end": result["energy_ratio_end"],
    "peak_freq": result["peak_freq"]
})
